In [2]:
def patient_generator():
    """
    Inputs : 
    All of the below
    
    Outputs : 
    - emergency patients 
    - elective patients 
    """
    
    def elective_patients():
        """ 
        Inputs : 
        - waiting list starting point
        - number of historical references
        - historical patients to select from
        - daycase:elective ratio
        
        Outputs :
        - forecasted number of patients
        - categories?
        - wait list
        """
        pass
    
    def emergency_patients():
        """
        Inputs : 
        - historical emergency patients
        - number of historical references
    
        Outputs :
        - forecasted number of emergencies
        - categories
        """
        pass
    
    pass

def theatre_and_beds():
    """
    Inputs : 
    - capacity of theatres
    - bed capacity
    
    Outputs : 
    - discharged patients
    Either (theatre capacity and bed capacity given)
    - theatre utilisation
    - bed utilisation
    Or (beds fixed)
    - number of theatres required to achieve 85% bed utilisation
    Or (theatre capacity fixed)
    - number of beds required to achieve maximum theatre capacity
    """
    # simply match patients to theatres and beds - theatre can't go ahead unless bed is free.
    pass

def metrics():
    """
    Inputs : 
    - simulation
    
    Outputs :
    - Bed utilisation
    - Theatre utilisation
    - Number of beds
    - Theatre capacity
    """
    pass

In [3]:
import sfttoolbox

import networkx as nx
import numpy as np

from dataclasses import dataclass, field

In [4]:
# Need SQL for:

# Theatre capacity
# Number of elective patients (per category?)(for forecasting)
# Historical elective patients - use to collect LoS?
# Number of emergency patients (per category?)(for forecasting)
# Historical emergency patients - use to collect LoS?
# Current day-case:elective ratio

# Patients starting point (emerg and elec)

In [5]:
## Taken from https://pythonhealthdatascience.github.io/intro-open-sim/lab/index.html

class Uniform:
    """
    Convenience class for the Uniform distribution.
    packages up distribution parameters, seed and random generator.
    """

    def __init__(self, low, high, random_seed=None):
        """
        Constructor

        Params:
        ------
        low: float
            lower range of the uniform

        high: float
            upper range of the uniform

        random_seed: int | SeedSequence, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.low = low
        self.high = high

    def sample(self, size=None):
        """
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.

        Returns:
        -------
        float or np.ndarray (if size >=1)
        """
        return self.rand.uniform(low=self.low, high=self.high, size=size)

In [9]:
# Probabilities would be filled in from the SQL/user, these are just an example currently
MINUTES_PER_HOUR = 60

iat_dist = Uniform(0, 24*MINUTES_PER_HOUR)

# This is just placeholder code!
######################################################################################
@dataclass
class Patient:
    id: int
    time_of_arrival: int
    pathway: list[str] = field(default_factory=list)


class PatientGenerator:
    def __init__(self, iat_dist):
        self.id = 0
        self.iat_dist = iat_dist
        
    # This is required to match the interface required for the simulation
    def generate_patients(self, day_num, day):
        patients = []
        
        num_patients = 10
        iats = self.iat_dist.sample(num_patients)
        
        # TODO: any iats over 24 would need to be pushed to the next day, this would impact the number of patients on a day though
        # TODO: split the emergency and elective patient generation
        
        # emergency patients
        for i in range(num_patients):
            patient = Patient(self.id, iats[i])
            patients.append(patient)
            self.id += 1
            
        # elective patients
        # for ...

        return patients
    
#####################################################################################

class Bed:
    def __init__(self, id):
        self.id = id
        self.occupied = False
        
    def occupy(self):
        self.occupied = True


class TheatreCapacity:
    def __init__(self, ward_capacity, critical_care_capacity):
        # Surgeries won't happen unless there's ward and cc capacity available..
        pass
    
    def get(self):
        # add to a wait list
        pass
    
    def update_day(self):
        pass

    
class BedCapacity:
    def __init__(self, num_beds=0):
        self.beds = [Bed(i) for i in range(num_beds)]
        # Use this to track beds at times throughout the day
        self.ward_movements = {}

    def check_for_free_beds(self):
        # TODO: Need to add in a time here, so beds are checked at a certain time
        for bed in self.beds:
            if not bed.occupied:
                return bed

    def get(self):
        # add to a wait list
        pass

    def update_day(self):
        pass        


ward_capacity = BedCapacity()
critical_care_capacity = BedCapacity(16)

theatre_capacity = TheatreCapacity(ward_capacity, critical_care_capacity)


# Distributions can take in patients, so probabilities could be created individually for patients
placeholder_dist = Uniform(0, 1)
@sfttoolbox.DES.distribution_wrapper
def patient_distribution():
    return placeholder_dist.sample(1)

G = nx.DiGraph()
G.add_edges_from([
    ("Patient arrives", "Emergency patient", {"probability":0.6}),
    ("Patient arrives", "Elective patient", {"probability":0.4}),
    
    ("Emergency patient", "Critical Care", {"probability":0.1}),
    ("Emergency patient", "Ward", {"probability":0.3}),
    ("Emergency patient", "Theatre", {"probability":0.6}),

    ("Elective patient", "Theatre", {"probability": 0.9}),
    ("Elective patient", "Daycase", {"probability": 0.1}),

    ("Daycase", "Ward", {"probability":0.6}),
    ("Daycase", "Discharge", {"probability":0.4}),

    ("Theatre", "Critical Care", {"probability":0.05}),
    ("Theatre", "Ward", {"probability":0.95}),

    ("Critical Care", "Ward", {"probability":0.3}),
    ("Critical Care", "Theatre", {"probability":0.7}),

    ("Ward", "Discharge"),
])

# TODO: These distributions are just placeholders for now
G.add_nodes_from([
    ("Patient arrives", {"distribution":patient_distribution}),
    ("Emergency patient", {"distribution":patient_distribution}),
    ("Elective patient", {"distribution":patient_distribution}),
    ("Daycase", {"distribution":patient_distribution}),
    ("Theatre", {"Capacity": theatre_capacity, "distribution":patient_distribution}),
    ("Critical Care", {"Capacity": critical_care_capacity, "distribution":patient_distribution}),
    ("Ward", {"Capacity": ward_capacity}),
])

# TODO: Add resources and distributions!


In [10]:
sim = sfttoolbox.DES.Simulation(G, PatientGenerator(iat_dist), None)

sim.plot_graph("theatre_simulation.html")

In [8]:
# TODO:
# Add in order to resolve in des framework for update_days?